WORKSHOP FINALE
===============

In [2]:
from pyplasm import *
import math
import csv

from larlib import *

In [3]:
def perimetro(): 
    with open("/home/gianlorenzo/PycharmProjects/ggpl/2017-12-11/perimetro.lines", "rb") as file:
        file_reader = csv.reader(file, delimiter=",")
        chiavi = {} ### diizionario delle chiavi
        nodi = [] ### array con i vertici
        archi= [] ### array con gli archi, ovvero gli spigoli
        i = 0 ### inizializzo un contatore
    
        for righe in file_reader:
            riga0 = math.floor(float(righe[0])*100)/100
            riga1 = math.floor(float(righe[1])*100)/100
            riga2 = math.floor(float(righe[2])*100)/100
            riga3 = math.floor(float(righe[3])*100)/100
        
            if not(riga0,riga1) in chiavi.keys():
                chiavi[(riga0,riga1)] = i
                nodi.append((riga0,riga1))
                i = i+1
            
            if not(riga2,riga3) in chiavi.keys():
                chiavi[(riga2,riga3)] = i
                nodi.append((riga2,riga3))
                i = i+1
            archi.append((chiavi[(riga0,riga1)],(chiavi[(riga2,riga3)])))
        pavimento = T(2)(450)(CUBOID([850,400]))
        corpoVilla = MULTEXTRUDE(OFFSET([20, 20])(STRUCT(MKPOLS([nodi,archi]))))(600)
    return (S([1,2,3])([3,3,3])(
        (STRUCT([pavimento,corpoVilla]))))

In [4]:
def arch(lateral_width, height, deep):

    _lateral_width = float(lateral_width)
    _height = float(height)
    _deep = float(deep)

    def bottomArc(d):
        return BEZIER(S1)([[0, 0], [0, 2 * d / 3], [d, 2 * d / 3], [d, 0]])

    def topArc(d):
        return BEZIER(S1)([[0, 2 * d / 3], [d, 2 * d / 3]])

    def arc2D(d):
        return BEZIER(S2)([bottomArc(d), topArc(d)])

    def arc3D(d):
        def arc3D1(w):
            arco = arc2D(d)
            dominio = PROD([INTERVALS(1)(8), INTERVALS(1)(1)])
            ar = MAP(arco)(dominio)
            domin = PROD([ar, QUOTE([1.5])])
            return COMP([T(2)(w), R([2, 3])(PI / 2)])(domin)

        return arc3D1

    def Interarc(d1, d2):
        def Interarc1(w):
            return CUBOID([d1, w, 2 * d2 / 3])

        return Interarc1

    def Xarc(d1, d2):
        def Xarc1(w):
            return RIGHT([RIGHT([Interarc(d1, d2)(w), arc3D(d2)(w)]), Interarc(d1, d2)(w)])

        return Xarc1

    return Xarc(_lateral_width, _height)(_deep)

In [5]:
def porticato():
    return STRUCT([CUBOID([540,650,1800]),T(1)(2040)(CUBOID([540,650,1800]))])

def arcate():
    return STRUCT([(S(1)(0.15)(T([1,2])([4230,750])(arch(15,2700,-50)))),
                  S(1)(0.2)(T([1,2])([5130,750])(arch(15,2700,-50))),
                  S(1)(0.2)(T([1,2])([7750,750])(arch(85,2700,-50)))])

def porticatoCompleto():
    return STRUCT([T(2)(700)(porticato()),T(1)(-90)(arcate())])

In [6]:
def tettoTriangolare2(depth,width):
    return ROTATE([1,2])(PI/2)(S([1,2,3])([28,28.65,35])(T([1,2,3])([93,-90,51])(ROTATE([1, 2])(PI)(ROTATE([2,3])(PI/2)(
        MULTEXTRUDE(MKPOL([[[0, 0],[width/2,width/4],[width,0]],[[1,2,3]],None]))(depth))))))

In [7]:
def tettoTriangolare1(depth,width):
    return TEXTURE("tetto.jpg")(S([1,2,3])([22,28.34,30])(T([1,2,3])([92.5,24,60])((ROTATE([1, 2])(PI)(ROTATE([2,3])(PI/2)(
        MULTEXTRUDE(MKPOL([[[0, 0],[width/2,width/4],[width,0]],[[1,2,3]],None]))(depth)))))))

In [8]:

def piramide(depth,width,height):
    
    _width = float(width)
    
    _scale = float(depth)/_width
    
    _height = float(height)

    return SCALE(2)(_scale)(ROTATE([1,2])(PI/4)(CONE([_width*(1/SQRT(2)),_height])(4)))

def scala():
    
    corrimanoDestro = TEXTURE("intonacoVilla.jpg")(T([1,2,3])([330,1070,127.5])(R([2,3])(-PI)(DIFFERENCE([CUBOID([20,150,130]),
                                                                             piramide(400,10000,100)]))))
    
    corrimanoSinistro = TEXTURE("intonacoVilla.jpg")(T([1,2,3])([500,1070,127.5])(R([2,3])(-PI)(DIFFERENCE([CUBOID([20,150,130]),
                                                                             piramide(400,10000,100)]))))
    
    return STRUCT([T([1,2,3])([350,1070,102])(R([2,3])(-PI)(TEXTURE("scala.jpg")(DIFFERENCE([CUBOID([160,150,103]),
                                                                        piramide(300,70000,100)])))),corrimanoDestro,
                                                                                                          corrimanoSinistro])

In [8]:
def villaGazzotti():
    return STRUCT([COLOR([0,0,0])(scala()),STRUCT([tettoTriangolare1(67.5,68),STRUCT([tettoTriangolare2(90,68),
                            STRUCT([porticatoCompleto(),(perimetro())])])])])

In [116]:
def comignoli():
    
    base = TEXTURE("intonacoVilla.jpg")(T([1,2])([2.5,2.5])(CUBOID([10,10,20])))
    
    tettino = TEXTURE("intonacoTetto.jpg")(T(3)(20)(CUBOID([15,15,2])))
    
    rialzo = TEXTURE("intonacoVilla.jpg")(CUBOID([2,1.5,2]))
    
    rialziDestra = TEXTURE("intonacoVilla.jpg")(
        STRUCT([T([1,2,3])([1,1,22])(rialzo),T([1,2,3])([1,6.75,22])(rialzo),T([1,2,3])([1,12.5,22])(rialzo)]))
    
    rialziSinistra = TEXTURE("intonacoVilla.jpg")(
        STRUCT([T([1,2,3])([12,1,22])(rialzo),T([1,2,3])([12,6.75,22])(rialzo),T([1,2,3])([12,12,22])(rialzo)]))
    
    tetto = TEXTURE("intonacoVilla.jpg")(T(3)(24)(CUBOID([15,15,1])))
    
    rialziTettoAvanti = STRUCT([R([1,3])(PI/4)((T([1,2,3])([15,0,17])(CUBOID([15,15,2])))),
                                R([1,3])(-PI/4)((T([1,2,3])([-18,0,28])(CUBOID([14,15,2]))))])
    
    comignolo = S([1,2,3])([9,9,9])(STRUCT([rialziTettoAvanti,
    base,tettino,rialziDestra,rialziSinistra,tettino,tetto]))
    
    return STRUCT([T([1,2,3])([265,850,625])(comignolo),T([1,2,3])([2255,850,625])(comignolo)])

In [118]:
VIEW(STRUCT([T(3)(1300)(comignoli()),villaGazzotti()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7fd1d634fc30> >

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7fd1d6353e40> >

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f06aa183750> >